In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import cv2
import albumentations as albu
from albumentations.pytorch import ToTensor

In [3]:
import PIL
print('PIL',PIL.__version__)

PIL 6.2.0


In [4]:
import torch
import torch.nn as nn

from torchvision import transforms

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
from Models.ResNet import ResNet, ResidualBlock

In [7]:
train = pd.read_csv("./input/train.csv")
test = pd.read_csv("./input/test.csv")

y = train["label"].values
X = train.drop("label", axis=1).values
test = test.values
    
X = X.reshape(-1, 28, 28, 1).astype(np.float32)
test = test.reshape(-1, 28, 28, 1).astype(np.float32)

#X = X.reshape(-1, 28, 28)
#test = test.reshape(-1, 28, 28)

train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.3)
    

In [8]:
train.shape

(42000, 785)

## Albumentation

In [9]:
class MNIST(torch.utils.data.Dataset):
    
    def __init__(self, images, labels, trans, flag=True):
        super().__init__()
        
        self.flag = flag
        self.trans = trans
        self.images = images
        self.labels = labels
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        
        if self.flag:
            image = self.images[idx]
            label = self.labels[idx]
            image = self.trans(image=image)['image']
            return image, label
        else:
            image = self.images[idx]
            image = self.trans(image=image)['image']
            return image

In [10]:
def train_augmentation():
    transform = [
        albu.Resize(32,32),
        albu.Normalize(mean=(0.485,), std=(0.229,)),
        ToTensor(),
    ]
    return albu.Compose(transform)

def test_augmentation():
    transform = [
        albu.Resize(32,32),
        albu.Normalize(mean=(0.485,), std=(0.229,)),
        ToTensor(),
    ]
    return albu.Compose(transform)

In [11]:
train_loader = torch.utils.data.DataLoader(MNIST(train_X, train_y, train_augmentation(), flag=True), batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(MNIST(val_X, val_y, train_augmentation(), flag=True), batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(MNIST(test, False, test_augmentation(), flag=False), batch_size=1, shuffle=False)

In [12]:
for image, label in train_loader:
    print(image.shape)
    break

torch.Size([64, 1, 32, 32])


# Models

In [13]:
net_args = {
    "block": ResidualBlock,
    "layers": [2, 2, 2, 2]
}

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
model = ResNet(**net_args).to(device)

# load

In [17]:
model.load_state_dict(torch.load("model.pth", map_location=device))

<All keys matched successfully>

# suiron

In [18]:
pred_list = []
with torch.no_grad():
    model.eval()
    c = 0
    for images in test_loader:
        images = images.to(device)
        output = model.forward(images)
        _, pred = torch.max(output.data, 1) 
        pred_list.append(pred.item())

In [19]:
pred_list = np.array(pred_list)
ID_lists= np.arange(1, pred_list.shape[0]+1) 
 
submissions=pd.DataFrame({"ImageId": ID_lists,
                         "Label": pred_list})
submissions.to_csv("submissions_eval.csv", index=False, header=True)

In [20]:
submissions.Label.value_counts()

1    3218
8    2869
0    2842
7    2827
2    2769
6    2761
4    2754
9    2720
3    2716
5    2524
Name: Label, dtype: int64

In [21]:
submissions

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
